In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt|
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("/content/flightPrice.csv")

In [ ]:
df

,Date_of_journey,Journey_day,Airline,Flight_code,Class,Source,Departure,Total_stops,Arrival,Destination,Duration_in_hours,Days_left,Fare
0,2023-01-16,Monday,SpiceJet,SG-8169,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5335
1,2023-01-16,Monday,Indigo,6E-2519,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.3333,1,5899
2,2023-01-16,Monday,GO FIRST,G8-354,Economy,Delhi,After 6 PM,non-stop,Before 6 AM,Mumbai,2.1667,1,5801
3,2023-01-16,Monday,SpiceJet,SG-8709,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.0833,1,5794
4,2023-01-16,Monday,Air India,AI-805,Economy,Delhi,After 6 PM,non-stop,After 6 PM,Mumbai,2.1667,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...,...
452083,2023-03-06,Monday,Vistara,UK-926,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,13.0833,50,65028
452084,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452085,2023-03-06,Monday,Vistara,UK-918,Business,Ahmedabad,Before 6 AM,1-stop,12 PM - 6 PM,Chennai,11.2500,50,69254
452086,2023-03-06,Monday,Vistara,UK-946,Business,Ahmedabad,6 AM - 12 PM,1-stop,After 6 PM,Chennai,11.1667,50,72980


In [ ]:
df.isnull().sum()

Date_of_journey      0
Journey_day          0
Airline              0
Flight_code          0
Class                0
Source               0
Departure            0
Total_stops          0
Arrival              0
Destination          0
Duration_in_hours    0
Days_left            0
Fare                 0
dtype: int64

In [ ]:
df['Date_of_journey'] = pd.to_datetime(df['Date_of_journey'], format='%Y-%m-%d')
df["Day"] = df["Date_of_journey"].dt.day
df["Month"] = df["Date_of_journey"].dt.month
df["Year"] = df["Date_of_journey"].dt.year
df.drop(columns=["Date_of_journey"], inplace=True)

In [ ]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [ ]:
columns_to_encode = ['Class', 'Source','Airline','Journey_day','Destination']

In [ ]:
one_hot_encoded_data = encoder.fit_transform(df[columns_to_encode])

In [ ]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

one_hot_encoded_df.index = df.index

df.drop(columns=columns_to_encode, inplace=True)

df = pd.concat([df, one_hot_encoded_df], axis=1)


In [ ]:
label_encoder = LabelEncoder()
df["Total_stops_encoded"] = label_encoder.fit_transform(df["Total_stops"])

df["Total_stops"] = df["Total_stops_encoded"]

In [ ]:
df.drop(columns=["Flight_code"], inplace=True)

In [ ]:
df["Departure"].unique()

array(['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM'],
      dtype=object)

In [ ]:
df["Arrival"].unique()

array(['After 6 PM', 'Before 6 AM', '6 AM - 12 PM', '12 PM - 6 PM'],
      dtype=object)

In [ ]:
Time_Slots = ['After 6 PM', 'Before 6 AM', '12 PM - 6 PM', '6 AM - 12 PM']

In [ ]:
encoder = OneHotEncoder(categories=[Time_Slots,Time_Slots], drop='first', sparse_output=False)

In [ ]:
one_hot_encoded_data = encoder.fit_transform(df[['Arrival', 'Departure']])

feature_names = encoder.get_feature_names_out(['Arrival', 'Departure'])

In [ ]:
one_hot_encoded_df = pd.DataFrame(one_hot_encoded_data, columns=feature_names)
one_hot_encoded_df.index = df.index

In [ ]:
df.drop(columns=['Arrival', 'Departure'], inplace=True)

In [ ]:
df = pd.concat([df, one_hot_encoded_df], axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452088 entries, 0 to 452087
Data columns (total 43 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Total_stops             452088 non-null  int64  
 1   Duration_in_hours       452088 non-null  float64
 2   Days_left               452088 non-null  int64  
 3   Fare                    452088 non-null  int64  
 4   Day                     452088 non-null  int32  
 5   Month                   452088 non-null  int32  
 6   Year                    452088 non-null  int32  
 7   Class_Economy           452088 non-null  float64
 8   Class_First             452088 non-null  float64
 9   Class_Premium Economy   452088 non-null  float64
 10  Source_Bangalore        452088 non-null  float64
 11  Source_Chennai          452088 non-null  float64
 12  Source_Delhi            452088 non-null  float64
 13  Source_Hyderabad        452088 non-null  float64
 14  Source_Kolkata      

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
y = df['Fare']
X = df.drop(columns=['Fare'])

In [ ]:
X = sm.add_constant(X)

In [ ]:
model = sm.OLS(y, X).fit()

In [ ]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 6.385e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        17:00:10   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452047   BIC:                         9.395e+06
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Total_stops            -1661

In [ ]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature          VIF
0              Total_stops          inf
1        Duration_in_hours     1.570360
2                Days_left   426.854725
3                      Day   176.346354
4                    Month   745.900733
5                     Year  4228.517933
6            Class_Economy     1.635916
7              Class_First     1.005208
8    Class_Premium Economy     1.511946
9         Source_Bangalore     2.306218
10          Source_Chennai     2.072468
11            Source_Delhi     2.454650
12        Source_Hyderabad     2.045468
13          Source_Kolkata     2.023243
14           Source_Mumbai     2.409069
15         Airline_AirAsia     1.189139
16        Airline_AkasaAir     1.072930
17     Airline_AllianceAir     1.011099
18        Airline_GO FIRST     1.135770
19          Airline_Indigo     1.835800
20        Airline_SpiceJet     1.060705
21         Airline_StarAir     1.002113
22         Airline_Vistara     1.693526
23      Journey_day_Monday     1.805282


In [ ]:
# Extract p-values
p_values = model.pvalues

In [ ]:
threshold = 0.05

In [ ]:
features_above_threshold = p_values[p_values > threshold]

In [ ]:
print("Features with p-value greater than 0.05:")
print(features_above_threshold)

Features with p-value greater than 0.05:
Airline_Indigo           0.079127
Airline_SpiceJet         0.063478
Journey_day_Monday       0.311049
Journey_day_Wednesday    0.592350
Destination_Delhi        0.944088
Arrival_12 PM - 6 PM     0.055597
dtype: float64


In [ ]:
X = X.drop(columns=['Destination_Delhi'])
X = X.drop(columns=['Journey_day_Wednesday'])
X = X.drop(columns=['Journey_day_Monday'])
X = X.drop(columns=['Airline_Indigo'])
X = X.drop(columns=['Airline_SpiceJet'])
X = X.drop(columns=['Arrival_12 PM - 6 PM'])

In the earlier file we checked the model summary after dropping each value one by one, and we the results.
So to make the results simpler let's just drop all 6 values at once and then check the overall effect then head to the VIF values.

In [ ]:
# Add a constant term to the feature matrix
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

In [ ]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 7.512e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        17:02:22   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452053   BIC:                         9.395e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Total_stops            -1662

Just like earlier we didnt see any changes in the R square value.

In [ ]:
# Set a threshold for VIF
vif_threshold = 10

# Identify features with VIF greater than the threshold
features_above_threshold = vif_data[vif_data["VIF"] > vif_threshold]

# Print the features with VIF greater than the threshold
print(f"Features with VIF greater than {vif_threshold}:")
print(features_above_threshold)

Features with VIF greater than 10:
                Feature          VIF
0           Total_stops          inf
2             Days_left   426.854725
3                   Day   176.346354
4                 Month   745.900733
5                  Year  4228.517933
35  Total_stops_encoded          inf


In [ ]:
# Check correlation between 'Total_stops' and 'Total_stops_encoded'
correlation_matrix = X[['Total_stops', 'Total_stops_encoded']].corr()
print("\nCorrelation between 'Total_stops' and 'Total_stops_encoded':")
print(correlation_matrix)


Correlation between 'Total_stops' and 'Total_stops_encoded':
                     Total_stops  Total_stops_encoded
Total_stops                  1.0                  1.0
Total_stops_encoded          1.0                  1.0


In [ ]:
# Decide to drop 'Total_stops_encoded' based on correlation
X_dropped = X.drop(columns=['Total_stops_encoded'])

In [ ]:
# Decide to drop 'Total_stops' based on correlation
X_dropped = X.drop(columns=['Total_stops'])

In [ ]:
model_dropped = sm.OLS(y, X_dropped).fit()
print("\nModel summary after dropping 'Total_stops_encoded':")
print(model_dropped.summary())


Model summary after dropping 'Total_stops_encoded':
                            OLS Regression Results                            
Dep. Variable:                   Fare   R-squared:                       0.850
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                 7.512e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):               0.00
Time:                        17:02:37   Log-Likelihood:            -4.6974e+06
No. Observations:              452088   AIC:                         9.395e+06
Df Residuals:                  452053   BIC:                         9.395e+06
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------

In [ ]:
# Identify features with high VIF
high_vif_features = vif_data[vif_data["VIF"] > 10]["Feature"].tolist()

In [ ]:
# Print features with high VIF values
print("\nFeatures with VIF greater than 10:")
print(vif_data[vif_data["VIF"] > 10])


Features with VIF greater than 10:
                Feature          VIF
0           Total_stops          inf
2             Days_left   426.854725
3                   Day   176.346354
4                 Month   745.900733
5                  Year  4228.517933
35  Total_stops_encoded          inf


In [ ]:
# Drop features with high VIF values
X_dropped = X.drop(columns=high_vif_features)

# Recalculate VIF after dropping the features
vif_data_dropped = pd.DataFrame()
vif_data_dropped["Feature"] = X_dropped.columns
vif_data_dropped["VIF"] = [variance_inflation_factor(X_dropped.values, i) for i in range(X_dropped.shape[1])]

# Print the VIF data after dropping the features
print("\nVIF data after dropping high VIF features:")
print(vif_data_dropped)


VIF data after dropping high VIF features:
                   Feature       VIF
0        Duration_in_hours  3.696286
1            Class_Economy  2.885569
2              Class_First  1.002638
3    Class_Premium Economy  1.714656
4         Source_Bangalore  1.908759
5           Source_Chennai  1.722220
6             Source_Delhi  2.135923
7         Source_Hyderabad  1.758137
8           Source_Kolkata  1.779886
9            Source_Mumbai  1.961719
10         Airline_AirAsia  1.114748
11        Airline_AkasaAir  1.028002
12     Airline_AllianceAir  1.005363
13        Airline_GO FIRST  1.079727
14         Airline_StarAir  1.001462
15         Airline_Vistara  2.657961
16    Journey_day_Saturday  1.290306
17      Journey_day_Sunday  1.290045
18    Journey_day_Thursday  1.294593
19     Journey_day_Tuesday  1.293097
20   Destination_Bangalore  1.577419
21     Destination_Chennai  1.459558
22   Destination_Hyderabad  1.485337
23     Destination_Kolkata  1.430448
24      Destination_Mumbai  1.6

In [ ]:
# Refit the model with the remaining features
model_dropped = sm.OLS(y, X_dropped).fit()

# Print the updated model summary
print("\nModel summary after dropping high VIF features:")
print(model_dropped.summary())



Model summary after dropping high VIF features:
                                 OLS Regression Results                                
Dep. Variable:                   Fare   R-squared (uncentered):                   0.850
Model:                            OLS   Adj. R-squared (uncentered):              0.850
Method:                 Least Squares   F-statistic:                          8.544e+04
Date:                Wed, 29 May 2024   Prob (F-statistic):                        0.00
Time:                        17:04:09   Log-Likelihood:                     -4.8815e+06
No. Observations:              452088   AIC:                                  9.763e+06
Df Residuals:                  452058   BIC:                                  9.763e+06
Df Model:                          30                                                  
Covariance Type:            nonrobust                                                  
                             coef    std err          t      P>|t|     

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y, test_size=0.2, random_state=42)

In [ ]:
# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=10, random_state=42)
rf_model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=10, random_state=42)

In [ ]:
# Make predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest Regression Model with 10 Estimators")
print(f"Mean Squared Error: {mse_rf}")
print(f"R-squared: {r2_rf}")

Random Forest Regression Model with 10 Estimators
Mean Squared Error: 36189166.51799265
R-squared: 0.9123041977135148


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Train the Gradient Boosting model with 10 estimators
gb_model = GradientBoostingRegressor(n_estimators=10, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions
y_pred_gb = gb_model.predict(X_test)

# Evaluate the model
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("\nGradient Boosting Regression Model with 10 Estimators")
print(f"Mean Squared Error: {mse_gb}")
print(f"R-squared: {r2_gb}")


Gradient Boosting Regression Model with 10 Estimators
Mean Squared Error: 106809503.56314184
R-squared: 0.741172676576181


In [ ]:
# Print the results for comparison
print("\nSummary of Model Performance")
print("Random Forest Regression with 10 Estimators")
print(f"Mean Squared Error: {mse_rf}")
print(f"R-squared: {r2_rf}")

print("\nGradient Boosting Regression with 10 Estimators")
print(f"Mean Squared Error: {mse_gb}")
print(f"R-squared: {r2_gb}")



Summary of Model Performance
Random Forest Regression with 10 Estimators
Mean Squared Error: 36189166.51799265
R-squared: 0.9123041977135148

Gradient Boosting Regression with 10 Estimators
Mean Squared Error: 106809503.56314184
R-squared: 0.741172676576181
